In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>")) # makes the notebook fill the whole window

In [40]:
import pandas as pd
import numpy as np
import re

import plotly.express as px

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import mean_squared_error

from yt_utils import YouTubeAccessor, YouTubeCategories

In [3]:
df = pd.read_feather("https://squeemos.pythonanywhere.com/static/yt_trending.feather")

In [4]:
sub = df.select_dtypes(include = [np.number])

In [5]:
sub

,snippet.thumbnails.default.width,snippet.thumbnails.default.height,snippet.thumbnails.medium.width,snippet.thumbnails.medium.height,snippet.thumbnails.high.width,snippet.thumbnails.high.height,snippet.thumbnails.standard.width,snippet.thumbnails.standard.height,snippet.categoryId,statistics.viewCount,statistics.likeCount,statistics.favoriteCount,statistics.commentCount,snippet.thumbnails.maxres.width,snippet.thumbnails.maxres.height,recordingDetails.location.latitude,recordingDetails.location.longitude,recordingDetails.location.altitude
0,120,90,320,180,480,360,640.0,480.0,24,3195647,348992.0,0,23723.0,NaN,NaN,NaN,NaN,NaN
1,120,90,320,180,480,360,640.0,480.0,1,19407491,NaN,0,NaN,1280.0,720.0,NaN,NaN,NaN
2,120,90,320,180,480,360,640.0,480.0,20,988800,47047.0,0,10395.0,1280.0,720.0,NaN,NaN,NaN
3,120,90,320,180,480,360,640.0,480.0,1,8043553,482862.0,0,20510.0,1280.0,720.0,NaN,NaN,NaN
4,120,90,320,180,480,360,640.0,480.0,24,4081570,189764.0,0,3645.0,1280.0,720.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13793,120,90,320,180,480,360,640.0,480.0,22,777297,64558.0,0,2545.0,1280.0,720.0,NaN,NaN,NaN
13794,120,90,320,180,480,360,640.0,480.0,24,397084,10986.0,0,2697.0,1280.0,720.0,NaN,NaN,NaN
13795,120,90,320,180,480,360,640.0,480.0,20,235878,20827.0,0,799.0,1280.0,720.0,NaN,NaN,NaN
13796,120,90,320,180,480,360,640.0,480.0,10,5628343,401639.0,0,26115.0,1280.0,720.0,NaN,NaN,NaN


In [6]:
def convert_pt_to_time(string):
    times = re.findall(r"\d+", string)
    if len(times) == 1:
        return int(times[0])
    elif len(times) == 2:
        minutes, seconds = times
        return int(minutes) * 60 + int(seconds)
    elif len(times) == 3:
        hours, minutes, seconds = times
        return int(hours) * 3600 + int(minutes) * 60 + int(seconds)
    else:
        raise Exception("More than 3 things trying to be unpacked")

In [7]:
df[df.yt.get_alias("duration")] = df.yt["duration"].apply(convert_pt_to_time)

In [9]:
mini_df = []
for vid_id in df.yt["id"].values:
    sub = df[df.yt["id"] == vid_id]
    sub = sub.sort_values(by = sub.yt.get_alias("queryTime"))
    mini_df.append(sub.iloc[-1])
mini_df = pd.DataFrame(mini_df)

In [10]:
px.histogram(
    mini_df,
    x = mini_df.yt.get_alias("duration")
)

In [32]:
x_data = np.array(mini_df.yt["duration"]).reshape(-1, 1)
y_data = mini_df.yt["viewCount"]

x_train, x_test, y_train, y_test = tts(x_data, y_data, test_size = .2)

In [36]:
model = SVC()
model = model.fit(x_train, y_train)
predictions = model.predict(x_test)

loss = mean_squared_error(predictions, y_test)
print(f"{loss:,}")

69,467,049,396,530.18


In [37]:
model = DecisionTreeRegressor()
model = model.fit(x_train, y_train)
predictions = model.predict(x_test)

loss = mean_squared_error(predictions, y_test)
print(f"{loss:,}")

11,710,201,002,049.059
